# 1 - Packages

In [ ]:
import time
import numpy as np
import pickle as pkl
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras

# 2 - Constants

In [ ]:
DATASET_PATH = '../../../dataset'
CHARS_NUM = 50
CLASSES_NUM = 15

# 3 - Load Data

In [ ]:
X_train = None
Y_train = None
with open(DATASET_PATH + '/X_train.pickle', 'rb') as X_train_file, \
         open(DATASET_PATH + '/Y_train.pickle', 'rb') as Y_train_file:
    X_train = pkl.load(X_train_file)
    Y_train = pkl.load(Y_train_file)
print('Training examples:', len(X_train))

X_val = None
Y_val = None
with open(DATASET_PATH + '/X_val.pickle', 'rb') as X_val_file, \
         open(DATASET_PATH + '/Y_val.pickle', 'rb') as Y_val_file:
    X_val = pkl.load(X_val_file)
    Y_val = pkl.load(Y_val_file)
print('Validation examples:', len(X_val))

X_test = None
Y_test = None
with open(DATASET_PATH + '/X_test.pickle', 'rb') as X_test_file, \
         open(DATASET_PATH + '/Y_test.pickle', 'rb') as Y_test_file:
    X_test = pkl.load(X_test_file)
    Y_test = pkl.load(Y_test_file)
print('Testing examples:', len(X_test))

# 4 - Model Structure

In [ ]:
def one_hot(input_dim=None, input_length=None):
    def _one_hot(x, classes_num):
        return keras.backend.one_hot(keras.backend.cast(x, 'uint8'),
                                     num_classes=classes_num)

    return keras.layers.Lambda(_one_hot,
                               arguments={'classes_num': input_dim},
                               input_shape=(input_length,))

def create_dense_layer(neurons):
    return keras.layers.Dense(neurons,
                              activation=tf.nn.relu,
                              kernel_initializer=keras.initializers.glorot_normal(seed=961))

def create_model(dropout_factor):
    model = tf.keras.models.Sequential([
        one_hot(83, 2 * CHARS_NUM),
        keras.layers.Flatten(),
        keras.layers.Dropout(dropout_factor),
        create_dense_layer(250),
        keras.layers.Dropout(dropout_factor),
        create_dense_layer(200),
        keras.layers.Dropout(dropout_factor),
        create_dense_layer(150),
        keras.layers.Dropout(dropout_factor),
        create_dense_layer(100),
        keras.layers.Dropout(dropout_factor),
        create_dense_layer(50),
        keras.layers.Dropout(dropout_factor),
        keras.layers.Dense(CLASSES_NUM,
                           activation=tf.nn.softmax,
                           kernel_initializer=keras.initializers.glorot_normal(seed=961))
    ])
    
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.train.AdamOptimizer(),
                  metrics=['accuracy'])
    
    return model

In [ ]:
model = create_model(0.025)
model.summary()

# 5 - Training

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, Y, batch_size):
        self.X, self.Y = X, Y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    def __getitem__(self, idx):
        X_batch = np.asarray(self.X[idx * self.batch_size:(idx + 1) * self.batch_size])
        Y_batch = np.asarray(self.Y[idx * self.batch_size:(idx + 1) * self.batch_size])
        
        X_tmp = list()
        for x in X_batch:
            before_need = x[0]
            after_need = x[-1]
            x_new = list()
            x_new.extend([1] * before_need)
            x_new.extend(x[1:-1])
            x_new.extend([1] * after_need)
            X_tmp.append(np.asarray(x_new))
        
        X_batch = np.asarray(X_tmp)
        Y_batch = np.asarray(Y_batch)
        
        return X_batch, Y_batch

In [ ]:
def fit_model(model, epochs, batch_size):
    checkpoint_path = 'checkpoints/epoch{epoch:02d}.ckpt'
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=0)
    
    training_generator = DataGenerator(X_train, Y_train, batch_size)
    val_generator = DataGenerator(X_val, Y_val, batch_size)

    model.fit_generator(generator=training_generator,
                        validation_data=val_generator,
                        epochs=epochs,
                        callbacks=[cp_callback])

In [ ]:
start_time = time.time()
fit_model(model, 50, 512)
end_time = time.time()
print("--- %s seconds ---" % round(end_time - start_time, 2))

# 6 - Evaluation

In [ ]:
test_generator = DataGenerator(X_test, Y_test, len(X_test))

XY_test = list()
for batch in range(len(test_generator)):
    XY_test.append(test_generator[batch])
X, Y = zip(*XY_test)

X = np.asarray(X)
Y = np.asarray(Y)
X = np.squeeze(X)
Y = np.squeeze(Y)

print(X.shape)
print(Y.shape)

In [ ]:
loss, acc = model.evaluate(X, Y, batch_size=512)
print('Accuracy: %s%%' % round(acc * 100, 2))
print('Loss: %s' % round(loss, 2))